In [131]:
import matplotlib.pyplot as pl
import numpy as np
import random
import qnm_filter
import qnm

In [137]:
def signal_creator(noise_scale):
    global signal, signal220, signal221, signalH_noise, t_range, mass
    mass_in_solar = random.uniform(50, 120)
    chi_inject = random.uniform(0.0, 0.95)
    injected = (mass_in_solar, chi_inject)
    mass = qnm_filter.Filter.mass_unit(injected[0])
    omega220 = qnm.modes_cache(s=-2,l=2,m=2,n=0)(a=injected[1])[0]
    omega221 = qnm.modes_cache(s=-2,l=2,m=2,n=1)(a=injected[1])[0]

    mmax = 8.4 * 1e-21
    phase1 = random.uniform(0, 2*np.pi)
    A220x = mmax*np.cos(phase1)
    A220y = mmax*np.sin(phase1)
    phase2 = random.uniform(0, 2*np.pi)
    A221x = mmax*np.cos(phase2)
    A221y = mmax*np.sin(phase2)

    amp220 = 1
    amp221 = 1

    sampling_frequency = 4096 * 1  # in Hz
    duration = 4  # in second
    t_range = np.arange(-duration / 2, duration / 2, 1 / sampling_frequency)
    signal220 = np.real(amp220 * (A220x + 1j * A220y) * np.exp(-1j * omega220 * np.abs(t_range / mass)))
    signal221 = np.real(amp221 * (A221x + 1j * A221y) * np.exp(-1j * omega221 * np.abs(t_range / mass)))
    
    bilby_ifo = qnm_filter.set_bilby_predefined_ifo(
    "H1", sampling_frequency, duration, start_time=-duration / 2
)
    signalH_noise = noise_scale*qnm_filter.bilby_get_strain(bilby_ifo, 0.0)
    signal = signal220+signal221+signalH_noise

In [138]:
def injection_evidence():
    fit = qnm_filter.Network(segment_length=0.2, srate=4096 * 1, t_init=3.0*mass)
    fit.original_data['H1'] = qnm_filter.RealData(signal, index=t_range)
    fit.detector_alignment()
    fit.pure_noise = {}
    fit.pure_noise['H1'] = qnm_filter.RealData(signalH_noise, index=t_range)
    fit.condition_data('original_data')
    fit.condition_data('pure_noise')
    fit.compute_acfs('pure_noise')
    fit.cholesky_decomposition()
    fit.first_index()
    
    fit.pure_nr = {}
    fit.pure_nr["H1"] = qnm_filter.RealData(signal220+signal221, index=t_range, ifo="H1")
    fit.condition_data('pure_nr')
    SNRtot_MF = fit.compute_SNR(
        fit.truncate_data(fit.original_data)["H1"],
        fit.truncate_data(fit.pure_nr)["H1"],
        "H1",
        False,
    )
    
    SNRtot_opt = fit.compute_SNR(
        None,
        fit.truncate_data(fit.pure_nr)["H1"],
        "H1",
        True,
    )
    return SNRtot_MF, SNRtot_opt

In [139]:
signal_creator(1)

In [140]:
noise_scale = 2
fit = qnm_filter.Network(segment_length=0.2, srate=4096 * 1, t_init=3.0*mass)
fit.original_data['H1'] = qnm_filter.RealData(signal220+signal221+signalH_noise*noise_scale, index=t_range)
fit.detector_alignment()
fit.pure_noise = {}
fit.pure_noise['H1'] = qnm_filter.RealData(signalH_noise*noise_scale, index=t_range)
fit.condition_data('original_data')
fit.condition_data('pure_noise')
fit.compute_acfs('pure_noise')
fit.cholesky_decomposition()
fit.first_index()

fit.pure_nr = {}
fit.pure_nr["H1"] = qnm_filter.RealData(signal220+signal221, index=t_range, ifo="H1")
fit.condition_data('pure_nr')
SNRtot_MF = fit.compute_SNR(
    fit.truncate_data(fit.original_data)["H1"],
    fit.truncate_data(fit.pure_nr)["H1"],
    "H1",
    False,
)
print(SNRtot_MF)

63.01386406333078


In [163]:
lower = 1
upper = 10
noise_space = np.power(np.linspace(1, 0.1, 150), -1)
temp_arr = []

for i in noise_space:
    signal_creator(i)
    temp_arr.append(injection_evidence())

temp_arr = np.asarray(temp_arr)

In [164]:
pl.hist(temp_arr[:,0], bins=20);